# Deploying pretrained VLM to FastAPI

### Integrating the pretrained VLM Model (Owl-Vit) into FastAPI

To aid in deployment, we would define the following functions and class

**FastAPI Application**
- test is a simple GET endpoint that returns a greeting.
- predict is a POST endpoint that takes a VLMInput object as input, performs object detection, and returns the bounding box coordinates of the detected objects.

1. **.Test Endpoint**
    - This endpoint is a simple test endpoint that returns a hello message with the item ID.
    - Request Method: GET
    - Request Body: None
    - Response: {"Hello": "World_{item_id}"}
    
2. **.od_predict - Object Detection Endpoint**
    - This endpoint takes an image URL or path and runs object detection on it using the finetuned model.
    - Request Method: POST
    - Request Body: VLMInput object with path_or_url and optional labels and threshold fields
    - Response: Object detection predictions in the format {"label": [(x1, y1, x2, y2), ...]}
    
3. **.clip_predict - CLIP Endpoint**
    - This endpoint takes an image URL or path and runs CLIP (Contrastive Language-Image Pre-training) on it with the given labels.
    - Request Method: POST
    - Request Body: VLMInput object with path_or_url and labels fields
    - Response: CLIP predictions in the format {"label": probability}
    
4. **owl_predict - OWL (Zero-Shot Object Detection) Endpoint**
    - This endpoint takes an image URL or path and runs OWL on it with the given labels and threshold.
    - Request Method: POST
    - Request Body: VLMInput object with path_or_url, labels, and threshold fields
    - Response: OWL predictions in the format {"label": [(x1, y1, x2, y2), ...]}

Note that we required the fine-tuned model for object detection from earlier units, please `run all` on the `1_Object_Detection_Fine_Tuning` notebook if you haven't run the notebook.

```python
import base64
from fastapi import FastAPI
from pydantic import BaseModel
import numpy as np
import io
from PIL import Image
import os
from transformers import pipeline
import urllib.request 
from torchvision.transforms import v2 as T
import torch
from urllib.request import urlretrieve
from transformers import CLIPProcessor, CLIPModel
from os import remove

app = FastAPI()

device = "cuda" if torch.cuda.is_available() else "cpu"

def load_image(path_or_url):
    """Loads an image from a given URL or path. If the input is a URL, 
    it downloads the image and saves it as a temporary file. If the input is a path,
    it loads the image from the path. The image is then converted to RGB format and returned."""
    if path_or_url.startswith('http'):  # assume URL if starts with http
        urlretrieve(path_or_url, "tmp.png")
        img = Image.open("tmp.png").convert("RGB")
        remove("tmp.png")  # cleanup temporary file
    else:
        img = Image.open(path_or_url).convert("RGB")
    return img

def ft_object_detection_predict(image):
    """Runs object detection on a given image using the fined tuned model that is safed locally. 
    The image is preprocessed, and the model is run on the device (either CPU or GPU). 
    The detections are then returned."""
    # helper function
    def get_transform(train):
        transforms = []
        if train:
            transforms.append(T.RandomHorizontalFlip(0.5))
        transforms.append(T.ToTensor())
        return T.Compose(transforms)
    
    model = torch.load('fine_tuned_OD_pedestrain_model.pth')
    eval_transform = get_transform(train=False)
    model.eval()
    with torch.no_grad():
        x = eval_transform(image)
        x = x[:3, ...].to(device)
        predictions = model([x, ])
        pred = predictions[0]
    return pred

def clip_predict(image, labels):
    """Runs CLIP (Contrastive Language-Image Pre-training) on a given image with the given labels. 
    The image and labels are preprocessed, and the model is run on the device (either CPU or GPU).
    The predictions are then returned."""
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    model.to(device)
    inputs = processor(text=labels.split(","), images=image, return_tensors="pt", padding=True)
    inputs.to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
    probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities
    return  {x:y.item() for x,y in zip(labels.split(","), probs[0])}

def owl_predict(img, labels, threshold):
    """Runs OWL (Zero-Shot Object Detection) on a given image with the given labels and threshold. 
    The image and labels are preprocessed, and the model is run on the device (either CPU or GPU). 
    The predictions are then filtered based on the threshold and returned."""
    checkpoint = "google/owlv2-base-patch16-ensemble"
    detector = pipeline(model=checkpoint, task="zero-shot-object-detection", device=0)
    predictions = detector(img,candidate_labels=labels.split(","))    
    predict_dict = {}
    for prediction in predictions:
        if prediction["score"]>threshold:
            label = prediction["label"]
            predict_dict[label] = [(prediction["box"]['xmin'], prediction["box"]['xmax'], prediction["box"]['ymin'], prediction["box"]['ymax'])]
    return predict_dict

class VLMInput(BaseModel):
    path_or_url: str
    labels: str = "None"
    threshold: float = 0.01
    
@app.get("/{item_id}")
def test():
    return {"Hello": f"World_{item_id}"}

@app.post("/od_predict")
async def predict(data: VLMInput):
    img = load_image(data.path_or_url)
    predict_dict = ft_object_detection_predict(img)
    normal_dict = {
        'boxes': predict_dict['boxes'].tolist(),
        'labels': predict_dict['labels'].tolist(),
        'scores': predict_dict['scores'].tolist()
    }
    return normal_dict

@app.post("/clip_predict")
async def predict(data: VLMInput):
    img = load_image(data.path_or_url)   
    predict_dict = clip_predict(img, data.labels)  
    return predict_dict

@app.post("/owl_predict")
async def predict(data: VLMInput):
    img = load_image(data.path_or_url)
    predict_dict = owl_predict(img, data.labels, data.threshold)
    return predict_dict


if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="0.0.0.0", port=8000)
```

### Create a Dockerfile
Create a `Dockerfile` in the same directory as your FastAPI app (`app.py`). This file will define the Docker image that includes your app and all its dependencies.

```docker
FROM us-docker.pkg.dev/deeplearning-platform-release/gcr.io/pytorch-gpu.2-2.py310

# Set the working directory in the container
WORKDIR /usr/src/app

COPY . /usr/src/app

# Install any needed packages specified in requirements.txt
RUN pip install --no-cache-dir -r requirements.txt

# Make port 8000 available to the world outside this container
EXPOSE 8000

# Define environment variable
ENV MODEL_PATH=/usr/src/app/models

# Run app.py when the container launches
CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]
```

### Create a Requirements File
Create a `requirements.txt` file that lists the packages that your app depends on. Make sure to include fastapi, uvicorn, torch, transformers, and any other required libraries. Torch isn't included in this `requirements.txt` because it's included in the starting Docker image (i.e. the image indicated in the first `FROM` line in the `Dockerfile`).

```txt
fastapi
uvicorn[standard]
pydantic
timm
transformers==4.37.0
accelerate
```


### Build the Docker Image
From your project directory (where your `Dockerfile` and `app.py` are located), run the following command to build the Docker image
```bash
docker build -t vlm_app .
```

### Run the Docker Container
```bash
docker run -p 8000:8000 --gpus all vlm_app

```

Docker runs the container and map port 8000 of the container to port 8000 on your host, allowing us to access the FastAPI application using the browser, `requests` library or Postman. We also give the container access to all the GPUs on our system such that it can run the models on GPU using CUDA, rather than on the CPU.

### Testing `vlm_app` using `requests`

In [43]:
import requests

# The endpoint URL
url = "http://localhost:8000/od_predict"

# Example url and context
data = {
    "path_or_url": "imgs/PennPed00048.png",
}

# Sending a POST request
response = requests.post(url, json=data)

# Print the response from the server
print("Status Code:", response.status_code)
print("Response:", response.json())

Status Code: 200
Response: {'boxes': [[265.0237121582031, 4.923355579376221, 388.3898010253906, 371.2566833496094], [112.45767211914062, 23.6782169342041, 201.7181854248047, 316.217041015625], [23.977638244628906, 20.964824676513672, 119.54765319824219, 319.9417419433594], [0.0, 75.43830871582031, 35.9254150390625, 310.942138671875], [43.246219635009766, 16.398618698120117, 184.44650268554688, 325.70440673828125]], 'labels': [1, 1, 1, 1, 1], 'scores': [0.9903967380523682, 0.9883424639701843, 0.9856445789337158, 0.46158814430236816, 0.33229777216911316]}


In [44]:
import requests

# The endpoint URL
url = "http://localhost:8000/clip_predict"

# Example url and context
data = {
    "path_or_url": "imgs/horserider.jpg",
    "labels": "horserider",
}

# Sending a POST request
response = requests.post(url, json=data)

# Print the response from the server
print("Status Code:", response.status_code)
print("Response:", response.json())

Status Code: 200
Response: {'horserider': 1.0}


In [45]:
import requests

# The endpoint URL
url = "http://localhost:8000/owl_predict"

# Example url and context
data = {
    "path_or_url": "https://th.bing.com/th/id/OIP.WhJW62tRiVMktCDMwRb52gHaJQ?rs=1&pid=ImgDetMain",
    "labels": "helmet, nasa badge",
}

# Sending a POST request
response = requests.post(url, json=data)

# Print the response from the server
print("Status Code:", response.status_code)
print("Response:", response.json())

Status Code: 200
Response: {'helmet': [[57, 176, 336, 493]], ' nasa badge': [[158, 250, 123, 168]]}


This result shows that the model is successfully able to respond to the request. 

# Exercise (20 mins)

**1. Object Detection: Refining Object Detection Results: Finding Bounding Box Centers**

Adjust the functions value to provide the center of the object's bounding box, you will need to modifying the app.py, rebuild the docker file and validate it with request.

**2. OWL-Vit: Refining Object Detection Results: Applying Confidence Thresholds (Bonus)** 

In object detection tasks, it's essential to refine the results to ensure accuracy and relevance. 
Let's explore how to modify the request to return only confident predictions